In [1]:
from os import sep, remove
import pickle
import pandas as pd
from datetime import date, datetime, timedelta
import utils
import requests
from bs4 import BeautifulSoup
import re
import nltk
import numpy as np


from num2words import num2words

import pickle
import weather
from sklearn import preprocessing
import scipy.sparse
import csv

In [2]:
pd.set_option('display.max_column', None)
pd.set_option('display.max_rows', None)

In [3]:
def get_date_from_user():
    #return datetime.combine(date.today() - timedelta(days=2),datetime.min.time())
    today = datetime.combine(date.today(),datetime.min.time())
    
    query = ""
    while query == "":
        try:
            query = datetime.strptime(input("Enter a date (YYYY-MM-DD): "), "%Y-%m-%d")
        except:
            print("Wrong format! Try again")
   
    while query < datetime(2023, 1, 20) or query > today:
        print("Failure!")
        try:
            query = datetime.strptime(input("Enter a date (YYYY-MM-DD): "), "%Y-%m-%d")
        except:
            print("Wrong format! Try again")
    time_query = ""
    while time_query == "":
        try:
            time_query = datetime.strptime(input("Enter time (HH:MM): "), "%H:%M")
        except:
            print("Wrong format! Try again")
    
    return datetime.combine(query.date(),time_query.time())
    


    
  

def remove_names_and_date(page_html_text):
    parsed_html = BeautifulSoup(page_html_text, features="html.parser")
    p_lines = parsed_html.findAll('p')
    
    min_sentense_word_count = 13
    p_index = 0
    
    #find first long sentense
    for p_line in p_lines:
        
        strong_lines = p_line.findAll('strong')
        if not strong_lines:
            p_index += 1
            continue

        for s in strong_lines:
            if len(s.text.split(" ")) >= min_sentense_word_count:
                break
        else:
            p_index += 1
            continue
        break
    for i in range(0, p_index):
        page_html_text = page_html_text.replace(str(p_lines[i]), "")
        
    return page_html_text 

      



def get_region_from_user():
    city = input("Enter the name of the region's capital (for example: Poltava or Полтава): ")
    while ((city not in df_regions["center_city_ua"].values) and (city not in df_regions["center_city_en"].values)):
        print("This is not a region's capital in Ukraine. Try again!")
        city = input("Enter the name of the region's capital (for example: Poltava or Полтава): ")
    index = df_regions.index[df_regions["center_city_en"] == city].tolist()[0] if city in df_regions["center_city_en"].values else df_regions.index[df_regions["center_city_ua"] == city].tolist()[0]
    return df_regions.iloc[index] #.at["center_city_en"]




def merge_weather_isw_region(df_weather, df_isw, df_region):
    #print(df_isw.shape)
    fields_to_exlude = [
    "city_resolvedAddress", 
    "day_datetime",
    "day_datetimeEpoch",
    "hour_datetime",
    "hour_datetimeEpoch",
    "city",
    "region",
    "center_city_ua",
    "center_city_en",
    "isw_report_date",
    "isw_date_tomorrow_datetime",
    "isw_text_main",
    "isw_keywords",
    "isw_data_lemmatized"
    ]
    df_isw["report_date"] = df_isw["date"].apply(lambda x: datetime.combine(x, datetime.min.time()))
    df_isw["date_tomorrow_datetime"] = df_isw["report_date"].apply(lambda x: (x + timedelta(days=1)))
    
    df_isw_short = df_isw[["report_date", "date_tomorrow_datetime", "keywords", "text_main", "data_lemmatized",]]
    weather_region_df = pd.merge(left=df_weather,right=df_region, left_on="city", right_on="center_city_ua")
    df_isw_short = df_isw_short.copy().add_prefix('isw_')
    df = weather_region_df.merge(df_isw_short,
                            how = "left",
                            left_on = "day_datetime", 
                            right_on = "isw_date_tomorrow_datetime")
    df['day_datetime'] = pd.to_datetime(df['day_datetime'])
    df_v2 = df.drop(fields_to_exlude, axis=1)
    short_df_region = df_region[["region_alt", "region_id"]]
    df_v2 = df_v2.merge(short_df_region, 
                            how = "left",   
                            left_on = "region_alt", 
                            right_on = "region_alt")
    df_v2["hour_conditions"] = df_v2["hour_conditions"].apply(lambda x: x.split(",")[0])
    
    label_encoder = pickle.load(open("model"+sep+"weather_conditions_label_encoder.pkl", "rb"))

    df_v2["hour_conditions_id"] = label_encoder.fit_transform(df_v2["hour_conditions"])
    tmp_fields_to_exlude = [
    "city_latitude",          
    "city_longitude",         
    "city_address",            
    "city_timezone",         
    "city_tzoffset",
    "day_feelslike",
    "day_feelslikemax",
    "day_feelslikemin",
    "day_sunrise", 
    "day_sunset", 
    "day_precipprob",
    "day_snow",
    "day_snowdepth",
    "day_windgust",
    "day_windspeed",
    "day_winddir",
    "day_pressure",
    "day_cloudcover",
    "day_visibility",
    "day_severerisk",
    "day_sunriseEpoch",
    "day_sunsetEpoch",
    "day_conditions",
    "day_description",
    "day_icon",
    "day_source",
    "day_preciptype",
    "day_stations",
    "hour_feelslike",
    "hour_preciptype",
    "hour_conditions",
    "region_alt",
    "hour_solarenergy",
    "hour_icon",
    "hour_source",
    "hour_stations"
    ]
    
    df_work_v3 = df_v2.drop(tmp_fields_to_exlude, axis=1).fillna(method="ffill")
    
    df_work_v3["region_id_x"] = df_work_v3["region_id_x"].apply(lambda x: int(x))
    df_work_v3["region_id_y"] = df_work_v3["region_id_y"].apply(lambda x: int(x))
    default_values = csv.DictReader(open("data"+sep+"weather_alarms_regions"+sep+"all_weather_by_hour_v2_mean_values.csv"))
    

    
    
    for i in default_values:
        df_work_v3.fillna(value=i, inplace=True)
        
    df_work_v3['day_solarradiation'] = df_work_v3['day_solarradiation'].astype('float64')
    df_work_v3['day_solarenergy'] = df_work_v3['day_solarenergy'].astype('float64')
    df_work_v3['day_uvindex'] = df_work_v3['day_uvindex'].astype('float64')
    df_work_v3['hour_visibility'] = df_work_v3['hour_visibility'].astype('float64')
    df_work_v3['hour_solarradiation'] = df_work_v3['hour_solarradiation'].astype('float64')
    df_work_v3['hour_uvindex'] = df_work_v3['hour_uvindex'].astype('float64')
    
    df_work_v3_csr = scipy.sparse.csr_matrix(df_work_v3.values)  
    
    
    word_count_vector = utils.cv.transform(df['isw_data_lemmatized'].values.astype('U'))
    
    with open(f"model"+sep+"word_count_vector_calculated.pkl", 'wb') as handle:
        pickle.dump(word_count_vector, handle)
    tfidf_vector = utils.tfidf.transform(word_count_vector)
    
    df_all_features = scipy.sparse.hstack((df_work_v3_csr, tfidf_vector), format='csr')
    return df_all_features





In [4]:
df_regions = pd.read_csv("data"+sep+"weather_alarms_regions"+sep+"regions.csv", sep=",")

chosen_date = get_date_from_user()

text_df = utils.get_text_df(chosen_date - timedelta(days=1))

text_df["date"] = chosen_date.date() - timedelta(days=1)
region_df = get_region_from_user()

our_weather = weather.get_weather_forecast(chosen_date.isoformat(), region_df.at["center_city_en"]+",UA")

Failure!
showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Danyil\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
region_df

region            Київська
center_city_ua        Київ
center_city_en        Kyiv
region_alt        Київщина
region_id               10
Name: 9, dtype: object

In [ ]:
our_weather

In [62]:
text_df

,date,title,text_title,text_main,main_html_v8,main_html,main_html1,main_html2,main_html3,main_html4,data_stemmed,data_lemmatized,keywords
0,2023-04-12,"Russian Offensive Campaign Assessment, April 1...","Russian Offensive Campaign Assessment, April 1...","<div class=""field-items""><div class=""field-ite...","\nKarolina Hird, Riley Bailey, Nicole Wolkov, ...","\nkarolina hird, riley bailey, nicole wolkov, ...","karolina hird, riley bailey, nicole wolkov, l...",karolina hird riley bailey nicole wolkov l...,karolina hird riley bailey nicole wolkov layn...,karolina hird riley bailey nicole wolkov layn...,karolina hird riley bailey nicol wolkov layn ...,karolina hird riley bailey nicole wolkov layn...,"{'april': 0.389, 'russification': 0.274, 'twel..."


In [65]:
text_df.loc[0, 'keywords']

{'april': 0.389,
 'russification': 0.274,
 'twelve': 0.194,
 'migrant': 0.175,
 'wagner': 0.158,
 'ideology': 0.153,
 'milbloggers': 0.143,
 'eleven': 0.141,
 'hear': 0.131,
 'court': 0.131,
 'minority': 0.117,
 'belarus': 0.115,
 'child': 0.113,
 'case': 0.111,
 'community': 0.103,
 'exercise': 0.099,
 'belarusian': 0.099,
 'kreminna': 0.095,
 'refusal': 0.094,
 'muted': 0.087,
 'bakhmut': 0.086,
 'ramification': 0.083,
 'domestic': 0.082,
 'near': 0.082,
 'kaluga': 0.079,
 'conscription': 0.075,
 'story': 0.074,
 'brutality': 0.074,
 'detailing': 0.072,
 'discrediting': 0.064,
 'ethnic': 0.063,
 'nationalist': 0.058,
 'xenophobia': 0.058,
 'tuvan': 0.058,
 'ironically': 0.058,
 'grodno': 0.058,
 'dodger': 0.058,
 'showing': 0.057,
 'law': 0.055,
 'registry': 0.055,
 'ground': 0.054,
 'source': 0.053,
 'space': 0.053,
 'official': 0.053,
 'improving': 0.053,
 'cracking': 0.053,
 'response': 0.052,
 'airborne': 0.052,
 'element': 0.051,
 'purportedly': 0.051,
 'onus': 0.051,
 'controve

In [68]:
weather_forecast_df = weather.vectorize(our_weather)
weather_forecast_df[0:5]

,city_latitude,city_longitude,city_resolvedAddress,city_address,city_timezone,city_tzoffset,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_feelslikemax,day_feelslikemin,day_feelslike,day_dew,day_humidity,day_precip,day_precipprob,day_precipcover,day_snow,day_snowdepth,day_windgust,day_windspeed,day_winddir,day_pressure,day_cloudcover,day_visibility,day_solarradiation,day_solarenergy,day_uvindex,day_severerisk,day_sunrise,day_sunriseEpoch,day_sunset,day_sunsetEpoch,day_moonphase,day_conditions,day_description,day_icon,day_source,day_preciptype,day_stations,hour_datetime,hour_datetimeEpoch,hour_temp,hour_feelslike,hour_humidity,hour_dew,hour_precip,hour_precipprob,hour_snow,hour_snowdepth,hour_preciptype,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_solarenergy,hour_uvindex,hour_severerisk,hour_conditions,hour_icon,hour_source,hour_stations
0,50.4506,30.5243,"Київ, Україна","Kyiv,UA",Europe/Kiev,3.0,2023-04-13,1681333200,14.6,8.6,11.2,14.6,8.6,11.1,8.4,83.3,0.1,100.0,4.17,0.0,0.0,19.8,7.9,48.8,1005.5,95.9,23.8,15.1,1.1,1.0,10.0,06:09:18,1681355358,19:48:50,1681404530,0.75,"Rain, Overcast",Cloudy skies throughout the day with early mor...,rain,comb,[rain],[remote],01:00:00,1681336800,9.7,9.0,94.12,8.8,0.0,0.0,0.0,0.0,[rain],16.9,6.5,82.3,1005.0,24.1,100.0,0.0,NaN,0.0,10.0,Overcast,cloudy,obs,[remote]
1,50.4506,30.5243,"Київ, Україна","Kyiv,UA",Europe/Kiev,3.0,2023-04-13,1681333200,14.6,8.6,11.2,14.6,8.6,11.1,8.4,83.3,0.1,100.0,4.17,0.0,0.0,19.8,7.9,48.8,1005.5,95.9,23.8,15.1,1.1,1.0,10.0,06:09:18,1681355358,19:48:50,1681404530,0.75,"Rain, Overcast",Cloudy skies throughout the day with early mor...,rain,comb,[rain],[remote],02:00:00,1681340400,9.5,8.8,94.11,8.6,0.1,100.0,0.0,0.0,[rain],16.6,6.1,84.5,1004.0,24.1,100.0,0.0,NaN,0.0,10.0,"Rain, Overcast",rain,obs,[remote]
2,50.4506,30.5243,"Київ, Україна","Kyiv,UA",Europe/Kiev,3.0,2023-04-13,1681333200,14.6,8.6,11.2,14.6,8.6,11.1,8.4,83.3,0.1,100.0,4.17,0.0,0.0,19.8,7.9,48.8,1005.5,95.9,23.8,15.1,1.1,1.0,10.0,06:09:18,1681355358,19:48:50,1681404530,0.75,"Rain, Overcast",Cloudy skies throughout the day with early mor...,rain,comb,[rain],[remote],03:00:00,1681344000,9.6,9.2,93.48,8.6,0.0,0.0,0.0,0.0,[rain],11.9,5.0,68.3,1004.0,24.1,97.4,0.0,NaN,0.0,10.0,Overcast,cloudy,obs,[remote]
3,50.4506,30.5243,"Київ, Україна","Kyiv,UA",Europe/Kiev,3.0,2023-04-13,1681333200,14.6,8.6,11.2,14.6,8.6,11.1,8.4,83.3,0.1,100.0,4.17,0.0,0.0,19.8,7.9,48.8,1005.5,95.9,23.8,15.1,1.1,1.0,10.0,06:09:18,1681355358,19:48:50,1681404530,0.75,"Rain, Overcast",Cloudy skies throughout the day with early mor...,rain,comb,[rain],[remote],04:00:00,1681347600,9.4,9.4,93.47,8.4,0.0,0.0,0.0,0.0,[rain],11.9,4.7,64.6,1004.0,24.1,91.0,0.0,NaN,0.0,10.0,Overcast,cloudy,obs,[remote]
4,50.4506,30.5243,"Київ, Україна","Kyiv,UA",Europe/Kiev,3.0,2023-04-13,1681333200,14.6,8.6,11.2,14.6,8.6,11.1,8.4,83.3,0.1,100.0,4.17,0.0,0.0,19.8,7.9,48.8,1005.5,95.9,23.8,15.1,1.1,1.0,10.0,06:09:18,1681355358,19:48:50,1681404530,0.75,"Rain, Overcast",Cloudy skies throughout the day with early mor...,rain,comb,[rain],[remote],05:00:00,1681351200,9.0,9.0,94.73,8.2,0.0,0.0,0.0,0.0,None,11.5,3.6,45.0,1004.0,24.1,95.8,0.0,NaN,0.0,10.0,Overcast,cloudy,obs,[remote]


In [69]:
weather_forecast_df["day_datetime"] = pd.to_datetime(weather_forecast_df["day_datetime"])
weather_forecast_df["city"] = weather_forecast_df["city_resolvedAddress"].apply(lambda x: x.split(",")[0])
weather_forecast_df["city"] = weather_forecast_df["city"].replace("Хмельницька область", "Хмельницький")

In [70]:
input_df = merge_weather_isw_region(weather_forecast_df, text_df, pd.DataFrame(region_df).transpose())

In [71]:
input_df

,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,day_solarenergy,day_uvindex,day_moonphase,hour_temp,hour_humidity,hour_dew,hour_precip,hour_precipprob,hour_snow,hour_snowdepth,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_uvindex,hour_severerisk,region_id_x,region_id_y,hour_conditions_id
0,14.6,8.6,11.2,8.4,83.3,0.1,4.17,15.1,1.1,1.0,0.75,9.7,94.12,8.8,0.0,0.0,0.0,0.0,16.9,6.5,82.3,1005.0,24.1,100.0,0.0,0.0,10.0,10,10,0
1,14.6,8.6,11.2,8.4,83.3,0.1,4.17,15.1,1.1,1.0,0.75,9.5,94.11,8.6,0.1,100.0,0.0,0.0,16.6,6.1,84.5,1004.0,24.1,100.0,0.0,0.0,10.0,10,10,2
2,14.6,8.6,11.2,8.4,83.3,0.1,4.17,15.1,1.1,1.0,0.75,9.6,93.48,8.6,0.0,0.0,0.0,0.0,11.9,5.0,68.3,1004.0,24.1,97.4,0.0,0.0,10.0,10,10,0
3,14.6,8.6,11.2,8.4,83.3,0.1,4.17,15.1,1.1,1.0,0.75,9.4,93.47,8.4,0.0,0.0,0.0,0.0,11.9,4.7,64.6,1004.0,24.1,91.0,0.0,0.0,10.0,10,10,0
4,14.6,8.6,11.2,8.4,83.3,0.1,4.17,15.1,1.1,1.0,0.75,9.0,94.73,8.2,0.0,0.0,0.0,0.0,11.5,3.6,45.0,1004.0,24.1,95.8,0.0,0.0,10.0,10,10,0
5,14.6,8.6,11.2,8.4,83.3,0.1,4.17,15.1,1.1,1.0,0.75,8.7,95.36,8.0,0.0,0.0,0.0,0.0,8.6,3.6,48.4,1004.0,24.1,94.3,0.0,0.0,10.0,10,10,0
6,14.6,8.6,11.2,8.4,83.3,0.1,4.17,15.1,1.1,1.0,0.75,8.6,95.36,7.9,0.0,0.0,0.0,0.0,9.0,3.6,30.7,1004.0,24.1,96.0,2.0,0.0,10.0,10,10,0
7,14.6,8.6,11.2,8.4,83.3,0.1,4.17,15.1,1.1,1.0,0.75,9.2,92.20,8.0,0.0,0.0,0.0,0.0,10.8,4.0,43.2,1005.0,24.1,80.0,10.0,0.0,10.0,10,10,1
8,14.6,8.6,11.2,8.4,83.3,0.1,4.17,15.1,1.1,1.0,0.75,10.3,86.23,8.1,0.0,0.0,0.0,0.0,14.0,4.7,29.9,1005.0,24.1,84.5,21.0,0.0,10.0,10,10,1
9,14.6,8.6,11.2,8.4,83.3,0.1,4.17,15.1,1.1,1.0,0.75,11.0,82.86,8.2,0.0,0.0,0.0,0.0,15.5,5.4,64.9,1005.0,24.1,95.0,16.0,0.0,10.0,10,10,0


In [72]:
df_work_v3_csr = scipy.sparse.csr_matrix(input_df.values)  

df = pd.DataFrame(np.tile(text_df, (24, 1)), columns=text_df.columns)

word_count_vector = utils.cv.transform(df['data_lemmatized'].values.astype('U'))

with open(f"model"+sep+"word_count_vector_calculated.pkl", 'wb') as handle:
    pickle.dump(word_count_vector, handle)
tfidf_vector = utils.tfidf.transform(word_count_vector)

df_all_features = scipy.sparse.hstack((df_work_v3_csr, tfidf_vector), format='csr')

In [73]:
df_all_features

<24x9032 sparse matrix of type '<class 'numpy.float64'>'
	with 19160 stored elements in Compressed Sparse Row format>

In [74]:
with open("model"+sep+"8_random_forest_v2.pkl", "rb") as modelfile:
    clf = pickle.load(modelfile, encoding="utf-8")
    schedule = clf.predict(df_all_features)


print("Prediction for the next 24 hours. \"0\" means there will be no alarm, \"1\" - there will be:")
print(schedule)

Prediction for the next 24 hours. "0" means there will be no alarm, "1" - there will be:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
